# Chapter 6 - Decision Trees

This notebook covers Decision Trees, including:
- Training and Visualizing Decision Trees
- Making Predictions
- Estimating Class Probabilities
- The CART Training Algorithm
- Computational Complexity
- Gini Impurity vs Entropy
- Regularization Hyperparameters
- Regression Trees
- Instability

## Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Training and Visualizing a Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

You can visualize the trained Decision Tree by first using the `export_graphviz()` method to output a graph definition file called *iris_tree.dot*:

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

In [ ]:
# Let's create a simple text representation
from sklearn.tree import export_text
r = export_text(tree_clf, feature_names=iris.feature_names[2:])
print(r)

## Making Predictions

In [ ]:
# Let's see how the tree makes predictions
print("Tree structure:")
print(f"Root node checks: petal length <= 2.45")
print(f"Left child (setosa): gini=0.0, samples=50")
print(f"Right child splits on: petal width <= 1.75")
print(f"Final prediction depends on the path taken")

# Test prediction
print(f"\nPrediction for sample with petal length=5, petal width=1.5:")
print(f"Predicted class: {tree_clf.predict([[5, 1.5]])}")
print(f"Predicted probabilities: {tree_clf.predict_proba([[5, 1.5]])}")

## Decision Tree Visualization

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

plt.figure(figsize=(8, 4))
plot_decision_boundary(tree_clf, X, y)
plt.plot([2.45, 2.45], [0, 3], "k-", linewidth=2)
plt.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.text(1.40, 0.8, "Depth=0\ngini=0.667\nsamples=150\nvalue=[50, 50, 50]", fontsize=12, bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.5))
plt.text(4.2, 1.2, "Depth=1\ngini=0.5\nsamples=100\nvalue=[0, 50, 50]", fontsize=12, bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.5))
plt.title("Decision Tree decision boundaries", fontsize=14)
save_fig("decision_tree_decision_boundaries_plot")
plt.show()

## Estimating Class Probabilities

In [ ]:
print(tree_clf.predict_proba([[5, 1.5]]))
print(tree_clf.predict([[5, 1.5]]))

## The CART Training Algorithm

Scikit-Learn uses the *Classification And Regression Tree* (CART) algorithm to train Decision Trees. The algorithm works by first splitting the training set in two subsets using a single feature $k$ and a threshold $t_k$ (e.g., "petal length ≤ 2.45 cm").

## Computational Complexity

Making predictions requires traversing the Decision Tree from the root to a leaf. Decision Trees are generally approximately balanced, so traversing the Decision Tree requires going through roughly $O(\log_2(m))$ nodes.

The training algorithm compares all features (or less if `max_features` is set) on all samples at each node. This results in a training complexity of $O(n × m \log(m))$.

## Gini Impurity or Entropy?

In [ ]:
# Compare Gini and Entropy
tree_clf_gini = DecisionTreeClassifier(max_depth=2, criterion="gini", random_state=42)
tree_clf_entropy = DecisionTreeClassifier(max_depth=2, criterion="entropy", random_state=42)

tree_clf_gini.fit(X, y)
tree_clf_entropy.fit(X, y)

plt.figure(figsize=(11, 4))

plt.subplot(121)
plot_decision_boundary(tree_clf_gini, X, y, legend=False)
plt.title("Gini", fontsize=14)

plt.subplot(122)
plot_decision_boundary(tree_clf_entropy, X, y, legend=True)
plt.title("Entropy", fontsize=14)

save_fig("gini_vs_entropy_plot")
plt.show()

Let's compare the impurity measures:

In [ ]:
# Plot Gini impurity and Entropy
def gini(p):
    return 1 - sum([pi**2 for pi in p])

def entropy(p):
    return -sum([pi * np.log2(pi) if pi > 0 else 0 for pi in p])

def classification_error(p):
    return 1 - max(p)

x = np.linspace(0.01, 0.99, 100)
gini_values = [gini([p, 1-p]) for p in x]
entropy_values = [entropy([p, 1-p]) for p in x]
error_values = [classification_error([p, 1-p]) for p in x]

plt.figure(figsize=(8, 6))
plt.plot(x, gini_values, "b-", linewidth=2, label="Gini")
plt.plot(x, entropy_values, "r--", linewidth=2, label="Entropy")
plt.plot(x, error_values, "g:", linewidth=2, label="Misclassification Rate")
plt.xlabel("Class 1 probability")
plt.ylabel("Impurity")
plt.legend(loc="upper right")
plt.title("Impurity Measures")
plt.grid(True)
save_fig("impurity_measures_plot")
plt.show()

## Regularization Hyperparameters

Decision Trees make very few assumptions about the training data. If left unconstrained, the tree structure will adapt itself to the training data, fitting it very closely, and most likely overfitting it.

In [ ]:
from sklearn.datasets import make_moons
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = DecisionTreeClassifier(random_state=42)
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("No restrictions", fontsize=14)
plt.sca(axes[1])
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("min_samples_leaf = 4", fontsize=14)
plt.ylabel("")
save_fig("min_samples_leaf_plot")
plt.show()

Other hyperparameters control the shape of the Decision Tree:

In [ ]:
# Demonstrate different regularization parameters
angle = np.pi / 180 * 20
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xr = X.dot(rotation_matrix)

tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(X, y)
tree_clf_sr.fit(Xr, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 3), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf_s, X, y, axes=[0.5, 7, 0, 3], iris=True)
plt.sca(axes[1])
plot_decision_boundary(tree_clf_sr, Xr, y, axes=[0.5, 7, 0, 3], iris=True)
plt.ylabel("")
save_fig("tree_instability_plot")
plt.show()

## Regression

In [ ]:
# Quadratic training set + noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_regression_predictions(tree_reg1, X, y)
for split, style in [(0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")]:
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.sca(axes[1])
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in [(0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")]:
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in [0.0458, 0.1298, 0.2873, 0.9040]:
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.title("max_depth=3", fontsize=14)

save_fig("tree_regression_plot")
plt.show()

## Tree Visualization with Export Text

In [ ]:
tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg1.fit(X, y)
print(export_text(tree_reg1))

## Regularization

In [ ]:
tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

x1 = np.linspace(0, 1, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

plt.sca(axes[0])
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("No restrictions", fontsize=14)

plt.sca(axes[1])
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf=10", fontsize=14)

save_fig("tree_regression_regularization_plot")
plt.show()

## Instability

In [ ]:
Xs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
ys = np.array([0, 0, 1, 1])

angle = np.pi / 4
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr.fit(Xsr, ys)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf_s, Xs, ys, axes=[-1.2, 1.2, -1.2, 1.2], iris=False)
plt.sca(axes[1])
plot_decision_boundary(tree_clf_sr, Xsr, ys, axes=[-1.2, 1.2, -1.2, 1.2], iris=False)
plt.ylabel("")
save_fig("sensitivity_to_rotation_plot")
plt.show()

## Feature Importance

In [ ]:
iris = load_iris()
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(iris.data, iris.target)

print("Feature importances:")
for name, score in zip(iris.feature_names, tree_clf.feature_importances_):
    print(f"{name}: {score:.3f}")

In [ ]:
# Visualize feature importance
plt.figure(figsize=(8, 6))
feature_names = iris.feature_names
importances = tree_clf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
plt.title("Feature Importances")
plt.tight_layout()
save_fig("feature_importance_plot")
plt.show()

## Summary

In this chapter, we covered:

1. **Decision Tree Fundamentals**: How trees make decisions using feature thresholds
2. **CART Algorithm**: The training process for classification and regression trees
3. **Impurity Measures**: Gini impurity vs entropy for splitting criteria
4. **Regularization**: Controlling overfitting with hyperparameters like max_depth, min_samples_leaf
5. **Regression Trees**: Applying decision trees to continuous target variables
6. **Instability**: Understanding sensitivity to data changes and rotations
7. **Feature Importance**: Quantifying which features are most useful

**Key Advantages:**
- Simple to understand and interpret
- Requires little data preparation
- Can handle both numerical and categorical data
- Can handle multi-output problems
- White box model (interpretable)
- Can validate using statistical tests

**Key Limitations:**
- Prone to overfitting (especially deep trees)
- Can be unstable (small data changes → different trees)
- Biased toward features with more levels
- Difficulty capturing linear relationships
- Can create overly complex trees

**Best Practices:**
- Use regularization parameters (max_depth, min_samples_leaf, etc.)
- Consider ensemble methods (Random Forest, Gradient Boosting)
- Validate using cross-validation
- Visualize trees to understand decision logic
- Check feature importance for insights